# concordance

> Functionality for concordance analysis.

In [ ]:
#| default_exp concordance

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
import time
import numpy as np
import polars as pl
import math
from fastcore.basics import patch
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets

In [ ]:
#| export
from conc.corpus import Corpus
from conc.result import Result
from conc.core import logger, PAGE_SIZE, EOF_TOKEN_STR, set_logger_state


In [ ]:
#| export
class Concordance:
	""" Class for concordancing. """
	def __init__(self,
			  corpus:Corpus # Corpus instance
			  ): 
		self.corpus = corpus


In [ ]:
#| exporti
@patch
def _get_concordance_sort(self:Concordance, 
						 token_positions: list[np.ndarray], # token index to get sort columns for
						 sort_columns: list # sort columns to use
						 ) -> tuple[np.ndarray, np.ndarray]: # token ids for first sort column and corresponding sort order
	""" Get the first sort column for a concordance. """

	start_time = time.time()
	index = 'orth_index'
	seq = np.array(token_positions[0]+sort_columns[0])
	sort_column_ids = self.corpus.get_tokens_by_index('orth_index')[seq]
	sort_column_order = self.corpus.token_ids_to_sort_order(sort_column_ids)
	logger.info(f'Concordance sort column ({sort_column_ids.shape[0]}) retrieval time: {(time.time() - start_time):.5f} seconds')
	return sort_column_ids, sort_column_order


In [ ]:
#| hide
import os

In [ ]:
#| hide
source_path = f'{os.environ.get("HOME")}/data/'
save_path = f'{os.environ.get("HOME")}/data/conc-test-corpora/'

path_to_toy_corpus = f'{save_path}toy.corpus'
path_to_brown_corpus = f'{save_path}brown.corpus'
path_to_reuters_corpus = f'{save_path}reuters.corpus'

In [ ]:
#| hide
toy = Corpus().load(path_to_toy_corpus)
report_toy = Concordance(toy)

In [ ]:
# load the corpus
brown = Corpus('brown').load(path_to_brown_corpus)

In [ ]:
# instantiate the Concordance class
report_brown = Concordance(brown)

In [ ]:
#| hide
token_str = 'dog'
token_sequence, index_id = brown.tokenize(token_str, simple_indexing=True)
token_positions = brown.get_token_positions(token_sequence, index_id)

sort_column_ids, sort_column_order = report_brown._get_concordance_sort(token_positions, [1, 2, 3])
print(sort_column_ids[:4])
print(brown.token_ids_to_tokens(sort_column_ids)[:4])
print(sort_column_order[:4])


[29064 38309 33838 15829]
['license' '.' 'owners' 'catchers']
[29512    40 36157  9356]


In [ ]:
#| export
@patch
def concordance(self: Concordance, 
				token_str: str, # token string to get concordance for 
				context_words:int = 5, # number of words to show on left and right of token string
				order:str='1R2R3R', # order of sort columns
				page_size:int=PAGE_SIZE, # number of results to display per results page
				page_current:int=1, # current page of results
				show_all_columns:bool = False, # df with all columns or just essentials
				use_cache:bool = True # retrieve the results from cache if available
				) -> Result: # concordance report results
	""" Report concordance for a token string. """

	# handled output from get concordance that returns columnar format rather than rowwise
	# shifted to polars dataframes 
	# make sure clean out Corpus.EOF_TOKEN from left and right
	# removed x*y iteration (is very slow) for getting tokens - apply vectorized method
	# TODO: improve ordering so not fixed options e.g. include 3R1R2R
	# TODO add in ordering by metadata columns or doc
	# DONE - reducing data retrieved to just the sort columns and then doing the concordance display separately here
	# DONE - could speed up the sort so that does a partial sort (e.g. just one or two columns) to get position of the slice - then handle ordering with smaller slice of data
	# e.g. if concordancing 'the' find what sort0 word is before start of that page and what word after - then return that slice and sort that slice only  
	# IDEA: potentially get sort columns until small enough result
	# TODO: look at retrieval of document_ids - use token2doc_index
	# TODO avoid any duplication related to retrieval of concordance vectors

	token_sequence, index_id = self.corpus.tokenize(token_str, simple_indexing=True)

	start_time = time.time()
	sequence_len = len(token_sequence[0])
	concordance_range = range(-1 * context_words, context_words + sequence_len)
	positional_columns = [str(x) for x in concordance_range]

	index = 'orth_index'

	cache_id = tuple(['concordance'] + list(token_sequence) + [order])

	if use_cache == True and cache_id in self.corpus.results_cache:
		logger.info('Using cached concordance results')
		positional_columns = self.corpus.results_cache[cache_id][0]
		concordance_df = self.corpus.results_cache[cache_id][1]
		total_count = self.corpus.results_cache[cache_id][2]
		total_docs = self.corpus.results_cache[cache_id][3]
		sort_columns = self.corpus.results_cache[cache_id][4]
	else:
		logger.info('Processing concordance results')
		token_positions = self.corpus.get_token_positions(token_sequence, index_id)

		if len(token_positions[0]) == 0:
			logger.info('No tokens found')
			return Result(type = 'concordance', df=pl.DataFrame(), title=f'Concordance for "{token_str}"', description=f'No matches', summary_data={}, formatted_data=[])

		if order == '1L2L3L':
			sort_columns = [-1,-2,-3]
		elif order == '3L2L1L':
			sort_columns = [-3,-2,-1]
		elif order == '2L1L1R':
			sort_columns = [-2,-1,sequence_len + 1 - 1]
		elif order == '1L1R2R':
			sort_columns = [-1,sequence_len + 1 - 1,sequence_len + 2 - 1]
		else:
			# i.e. 1R2R3R
			sort_columns = [sequence_len + 1 - 1,sequence_len + 2 - 1,sequence_len + 3 - 1]

		# getting first sort column here
		sort_column_ids, sort_column_order = self._get_concordance_sort(token_positions, sort_columns)
		
		concordance_df = pl.DataFrame([pl.Series(name='index', values=token_positions[0]), pl.Series(name='sort0', values=sort_column_order), pl.Series(name=str(sort_columns[0]), values=sort_column_ids)])
		concordance_df = concordance_df.sort('sort0')
		concordance_df = concordance_df.with_row_index('row')

		total_count = len(concordance_df)
		total_docs = len(np.unique(self.corpus.get_tokens_by_index('token2doc_index')[np.array(token_positions[0])])) # REFACTORED - was using old self.corpus.token2doc_index

		self.corpus.results_cache[cache_id] = [positional_columns, concordance_df, total_count, total_docs, sort_columns]

	# working out relevant slice to populate 
	resultset_start = page_size*(page_current-1)
	resultset_len = page_size
	resultset_end = min(resultset_start + resultset_len, len(concordance_df) - 1)
	
	start_order = concordance_df['sort0'][resultset_start]
	end_order = concordance_df['sort0'][resultset_end]
	start_order_pos = concordance_df.filter(pl.col("sort0") == start_order).head(1)['row'].item()
	end_order_pos = concordance_df.filter(pl.col("sort0") == end_order).tail(1)['row'].item()
	
	# populating a smaller chunk of the concordance report - as only need to retrieve/sort a subset
	concordance_result_df = concordance_df.slice(start_order_pos, end_order_pos - start_order_pos + 1)

	results_start_time = time.time()
	concordance_columns = []
	seq = concordance_result_df['index'].to_numpy()
	for pos in concordance_range:
		tokens = self.corpus.get_tokens_by_index(index)[np.array(seq+pos)] # REFACTORED - was using getattr call to get orth_index here
		concordance_columns.append(pl.Series(name=str(pos), values=tokens))
		if pos in sort_columns:
			column_name = 'sort'+str(sort_columns.index(pos))
			if column_name != 'sort0':
				concordance_columns.append(pl.Series(name=column_name, values=self.corpus.token_ids_to_sort_order(tokens)))
	logger.info(f'Concordance results ({len(concordance_columns[0])}) retrieval time: {(time.time() - results_start_time):.5f} seconds')

	concordance_result_df = concordance_result_df.with_columns(concordance_columns)
	#offsets_arr = np.array(self.corpus.offsets,dtype=np.uint64) # FIX
	#document_ids = np.searchsorted(offsets_arr, concordance_result_df['index'], side = 'right') - 1 
	document_ids = self.corpus.get_tokens_by_index('token2doc_index')[np.array(concordance_result_df['index'])] # REFACTORED to remove offsets functionality
	concordance_result_df = concordance_result_df.with_columns(pl.Series(name="document_id", values=document_ids))
	concordance_result_df = concordance_result_df.sort(['sort0','sort1','sort2'])
		
	# slicing this further to get only the required page of results and then populating with left, keyword, right strings
	concordance_view_df = concordance_result_df.slice(start_order_pos - resultset_start, page_size)

	concordance_left = []
	concordance_right = []
	concordance_keyword = []

	for pos in positional_columns:
		if int(pos) < 0:
			concordance_left.append(self.corpus.token_ids_to_tokens(concordance_view_df[str(pos)].to_numpy()))
		elif int(pos) == 0 or int(pos) < sequence_len:
			concordance_keyword.append(self.corpus.token_ids_to_tokens(concordance_view_df[str(pos)].to_numpy()))
		else:
			concordance_right.append(self.corpus.token_ids_to_tokens(concordance_view_df[str(pos)].to_numpy()))

	concordance_left = [(' '.join(column)).split(EOF_TOKEN_STR)[-1] for column in np.array(concordance_left).T]
	concordance_keyword = [' '.join(column) for column in np.array(concordance_keyword).T]
	concordance_right = [(' '.join(column)).split(EOF_TOKEN_STR)[0] for column in np.array(concordance_right).T]

	concordance_view_df = concordance_view_df.with_columns(pl.Series(name='left', values=concordance_left), pl.Series(name='node', values=concordance_keyword), pl.Series(name='right', values=concordance_right))

	total_pages = math.ceil(total_count/page_size)
	summary_data = {'total_count': total_count, 'total_docs': total_docs, 'page': page_current, 'total_pages': total_pages}
	formatted_data = [f'Total Concordance Lines: {total_count}', f'Total Documents: {total_docs}', f'Showing {min(page_size, total_count)} lines', f'Page {page_current} of {total_pages}']

	if show_all_columns == False:
		concordance_view_df = concordance_view_df[['document_id', 'left', 'node', 'right']]
	
	logger.info(f'Concordance report time: {(time.time() - start_time):.5f} seconds')

	return Result(type = 'concordance', df=concordance_view_df, title=f'Concordance for "{token_str}"', description=f'Context tokens: {context_words}, Order: {order}', summary_data=summary_data, formatted_data=formatted_data)


In [ ]:
#| hide
# no result
assert report_toy.concordance('dsahjhdsjhdsa', context_words=5).df.select(pl.len()).item() == 0

In [ ]:
#| hide
report_toy.concordance('the', context_words=5).display()

Concordance for "the" 
 
 
 Context tokens: 5, Order: 1R2R3R 
 
 
 Document Id 
 Left 
 Node 
 Right 
 
 
 
 
 5 
 
 The 
 cat is climbing a tree 
 
 
 3 
 
 The 
 cat is meowing . 
 
 
 1 
 
 The 
 cat sat on the mat 
 
 
 4 
 
 The 
 dog is barking . 
 
 
 6 
 
 The 
 dog is digging a hole 
 
 
 2 
 
 The 
 dog sat on the mat 
 
 
 1 
 The cat sat on 
 the 
 mat . 
 
 
 2 
 The dog sat on 
 the 
 mat . 
 
 
 
 
 
 Total Concordance Lines: 8 
 


 
 Total Documents: 6 
 


 
 Showing 8 lines 
 


 
 Page 1 of 1

In [ ]:
report_brown.concordance('good at', context_words = 10, order='1R2R3R').display()

Concordance for "good at" 
 
 
 Context tokens: 10, Order: 1R2R3R 
 
 
 Document Id 
 Left 
 Node 
 Right 
 
 
 
 
 484 
 about twenty miles away , and he was also pretty 
 good at 
 anything in the carpentry line . was a vivid , 
 
 
 263 
 he says , ' as a storyteller and was precociously 
 good at 
 description , dialogue , and most of the other staples 
 
 
 479 
 and not a method of passing the day . was 
 good at 
 his job . probably was n't hard for him to 
 
 
 474 
 trying to flatter her vanity . You must have been 
 good at 
 history at school . did you go to school '' 
 
 
 82 
 enough of unequal merit , but all of them pretty 
 good at 
 that . consisted of a new arrangement of ` ` 
 
 
 474 
 Why not '' ? ? said . I 'm not 
 good at 
 that kind of thing '' . This afternoon let 's 
 
 
 
 
 
 Total Concordance Lines: 6 
 


 
 Total Documents: 5 
 


 
 Showing 6 lines 
 


 
 Page 1 of 1

In [ ]:
#| hide
congress = Corpus().load(f'{save_path}us-congressional-speeches-subset-100k.corpus')
report_congress = Concordance(congress)
congress._init_token_arrays()

In [ ]:
#| hide
%time report_congress.concordance('free', context_words = 5, order='1R2R3R').display()

Concordance for "free" 
 
 
 Context tokens: 5, Order: 1R2R3R 
 
 
 Document Id 
 Left 
 Node 
 Right 
 
 
 
 
 16,240 
 camp inscribed " Work makes 
 Free 
 " . the ones who 
 
 
 89,400 
 emphasized the distinction between " 
 free 
 " and " fair " 
 
 
 53,584 
 " after the word " 
 free 
 " and before the word 
 
 
 53,050 
 the comment about " rent 
 free 
 " because I will show 
 
 
 73,211 
 We inserted the word " 
 free 
 " in the other relation 
 
 
 41,763 
 the opportunity to obtain " 
 free 
 " money . from Washington 
 
 
 83,010 
 an effort to supposedly " 
 free 
 " the Salvadoran people of 
 
 
 53,584 
 therefore . the word " 
 free 
 " would exclude college libraries 
 
 
 50,033 
 we all could use a 
 free 
 $ 5.000 . farmers now 
 
 
 94,185 
 lie utterly impiacticableto allow the 
 free 
 ( oming and going of 
 
 
 26,114 
 . 737.92 Porto Rico . 
 free 
 ----- 469 . 205 . 
 
 
 78,059 
 the country needs is the 
 free 
 -and unlimited coinage of silver 
 
 
 30,742 
 is the " fair and 
 free 
 -election " that the gentleman 
 
 
 21,592 
 - . Recently the Detroit 
 Free 
 -Press . pub : lished 
 
 
 86,957 
 if they will still be 
 free 
 . 
 
 
 
 35,515 
 . " He goes scot 
 free 
 . 
 
 
 
 18,153 
 and monumental efforts to be 
 free 
 . 
 
 
 
 19,414 
 They may have their alcohol 
 free 
 . 
 
 
 
 36,360 
 that Poland will again be 
 free 
 . 
 
 
 
 73,356 
 want them to come In 
 free 
 . 
 
 
 
 
 
 
 Total Concordance Lines: 4643 
 


 
 Total Documents: 3229 
 


 
 Showing 20 lines 
 


 
 Page 1 of 233

CPU times: user 937 ms, sys: 334 ms, total: 1.27 s
Wall time: 656 ms


In [ ]:
#| hide
congress.tokenize('government', simple_indexing=True)

([(np.int64(60063),)], 66)

In [ ]:
#| export
@patch
def concordance_plot(self: Concordance,
						token_str: str, # string to get concordance for 
						page_size:int=8 # number of results to display per results page
	): # concordance plot
	""" Display concordance plot. """

	token_sequence, index_id = self.corpus.tokenize(token_str, simple_indexing=True)
	token_positions = self.corpus.get_token_positions(token_sequence, index_id)

	if len(token_positions[0]) > 0:

		document_ids = self.corpus.get_tokens_by_index('token2doc_index')[token_positions[0]]
		unique_document_ids = np.unique(document_ids)
		num_docs = len(unique_document_ids)

	font_family = "'Segoe UI', Roboto, Oxygen, Ubuntu, Cantarell, 'Helvetica Neue', 'Fira Sans', 'Droid Sans', Arial, sans-serif;"

	plots_per_page = 8
	num_pages = int(np.ceil(num_docs / plots_per_page))
	per_subplot_height = 50
	rows = plots_per_page 

	fig = make_subplots(
		rows=rows,
		cols=1,
		vertical_spacing=0.05,
		subplot_titles=[None]*rows,
	)

	for idx in range(plots_per_page):
		row = idx + 1

		fig.add_shape(
			type="rect",
			x0=0, y0=0, x1=100, y1=1,
			line=dict(color="black", width=1),
			fillcolor="gray",
			opacity=0.08,
			layer="below",
			row=row, col=1
		)

		fig.update_xaxes(range=[0, 100], visible=False, row=row, col=1)
		fig.update_yaxes(range=[0, 1], visible=False, row=row, col=1)
		yref = "y" if row == 1 else f"y{row} domain"

		# fig.add_annotation(
		# 	text=f"Doc {i}",
		# 	xref="paper", yref=yref,
		# 	x=-0.03, y=0.6,
		# 	showarrow=False,
		# 	xanchor="right",
		# 	yanchor="middle",
		# 	font=dict(size=12, family = font_family)
		# )

		# fig.add_annotation(
		# 	text=f"{len(normalized_documents[i])} lines",
		# 	xref="paper", yref=yref,
		# 	x=-0.03, y=0.25, 
		# 	showarrow=False,
		# 	xanchor="right",
		# 	yanchor="middle",
		# 	font=dict(size=11, color="gray", family = font_family)
		# )

	fig.update_xaxes(fixedrange=True, row=row, col=1)
	fig.update_yaxes(fixedrange=True, row=row, col=1)

	fig.update_layout(
		height=per_subplot_height * plots_per_page,
		width=600,
		showlegend=False,
		title_text=f"Concordance Plot", #Page {page} of {num_pages}
		margin=dict(t=40, b=20, l=80, r=20),
		font=dict(family=font_family, size=12),
	)

	#fig.show(config={'displaylogo': False})
	fig_widget = go.FigureWidget(fig)

	page_slider = widgets.IntSlider(min=1, max=num_pages, step=1, description='Page', layout=widgets.Layout(width='600px'))
	display(page_slider)
	display(fig_widget)

	# def plot_page(page):


		# doc_range = self.corpus.tokens.with_row_index('position').filter(pl.col('token2doc_index').is_in(unique_document_ids)).group_by('token2doc_index').agg([pl.col('position').min().alias('min'), pl.col('position').max().alias('max')]).collect()

		# documents = [ [] for _ in range(num_docs) ]  
		# normalized_documents = [ [] for _ in range(num_docs) ] 
		# for i, doc_id in enumerate(document_ids):
		# 	doc_index = np.where(unique_document_ids == doc_id)[0][0]
		# 	documents[doc_index].append(token_positions[0][i])
		# 	position_min, position_max = doc_range.filter(pl.col('token2doc_index') == doc_id).select(['min', 'max']).to_numpy()[0]
		# 	normalized_documents[doc_index].append((token_positions[0][i] - position_min) / (position_max - position_min) * 100)  # normalizing the position to be between 0 and 100

	# 	start = (page - 1) * plots_per_page
	# 	end = min(start + plots_per_page, num_docs)
	# 	n_plots_this_page = end - start

	# 	for idx in range(plots_per_page):
	# 		row = idx + 1
	# 		if idx < n_plots_this_page:
	# 			i = start + idx
	# 			for x0 in normalized_documents[i]:
	# 				fig.add_trace(
	# 					go.Scatter(
	# 						x=[x0, x0],
	# 						y=[0, 1],
	# 						mode='lines',
	# 						line=dict(color='black', width=2),
	# 						showlegend=False,
	# 						hoverinfo='text',
	# 						hovertext=f"Vertical at x = {x0:.2f}"
	# 					),
	# 					row=row, col=1
	# 				)
	# 		else:
	# 			fig.update_xaxes(visible=False, row=row, col=1)
	# 			fig.update_yaxes(visible=False, row=row, col=1)


	def update(page):
		logger.info(page)
		start = (page - 1) * plots_per_page
		end = min(start + plots_per_page, num_docs)
		n_plots_this_page = end - start

		doc_range = self.corpus.tokens.with_row_index('position').filter(pl.col('token2doc_index').is_in(unique_document_ids)).group_by('token2doc_index').agg([pl.col('position').min().alias('min'), pl.col('position').max().alias('max')]).collect()

		documents = [ [] for _ in range(num_docs) ]  
		normalized_documents = [ [] for _ in range(num_docs) ] 
		for i, doc_id in enumerate(document_ids):
			doc_index = np.where(unique_document_ids == doc_id)[0][0]
			documents[doc_index].append(token_positions[0][i])
			position_min, position_max = doc_range.filter(pl.col('token2doc_index') == doc_id).select(['min', 'max']).to_numpy()[0]
			normalized_documents[doc_index].append((token_positions[0][i] - position_min) / (position_max - position_min) * 100)  # normalizing the position to be between 0 and 100

		for i, positions in enumerate(normalized_documents):
			for position in positions:
				i = start + idx
				for x0 in normalized_documents[i]:
					fig_widget.add_trace(
						go.Scatter(
							x=[x0, x0],
							y=[0, 1],
							mode='lines',
							line=dict(color='black', width=2),
							showlegend=False,
							hoverinfo='text',
							hovertext=f"Vertical at x = {x0:.2f}"
						),
						row=row, col=1
					)

	update(1)

	page_slider.observe(update, names='value')

	# _ = widgets.interact(plot_page, page=page_slider, style={'font_family': font_family})



In [ ]:
set_logger_state('verbose')
report_brown.concordance_plot('peaceful')

2025-06-03 20:53:50 - INFO - tokenize - Tokenization time: 0.00024 seconds
2025-06-03 20:53:50 - INFO - get_token_positions - Token indexing (26) time: 0.00288 seconds


IntSlider(value=1, description='Page', layout=Layout(width='600px'), max=3, min=1)

FigureWidget({
    'data': [],
    'layout': {'font': {'family': ("'Segoe UI', Roboto, Oxygen, Ub" ... "roid Sans', Arial, sans-serif;"),
                        'size': 12},
               'height': 400,
               'margin': {'b': 20, 'l': 80, 'r': 20, 't': 40},
               'shapes': [{'fillcolor': 'gray',
                           'layer': 'below',
                           'line': {'color': 'black', 'width': 1},
                           'opacity': 0.08,
                           'type': 'rect',
                           'x0': 0,
                           'x1': 100,
                           'xref': 'x',
                           'y0': 0,
                           'y1': 1,
                           'yref': 'y'},
                          {'fillcolor': 'gray',
                           'layer': 'below',
                           'line': {'color': 'black', 'width': 1},
                           'opacity': 0.08,
                           'type': 'rect',
           

2025-06-03 20:53:50 - INFO - update - 1


TypeError: unsupported operand type(s) for -: 'Bunch' and 'int'

In [ ]:
#| hide
# congress.tokens.with_row_index('position').filter(pl.col('lower_index') == 60063).head(21).collect()

In [ ]:
#| hide
# TODO - evaluate potential improvement to concordance to reduce data-assembly with large concordance result-sets
# def populate_concordance_result(self:Concordance, seq, concordance_range, sort_columns, index='orth_index'):
# 	""" Get the concordance results from a slice of the concordance dataframe. """

# 	tokens = getattr(self.corpus, index)
# 	concordance_columns = []
# 	concordance_columns.append(pl.Series(name='sort1', values=self.corpus.token_ids_to_sort_order(tokens[seq+sort_columns[1]])))
# 	concordance_columns.append(pl.Series(name='sort2', values=self.corpus.token_ids_to_sort_order(tokens[seq+sort_columns[2]])))
	
# 	return concordance_columns


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()